![Иллюстрация сплита](/img/gts_split.png)

### Описание сплита

__Цель сплита:__ смоделировать онлайн-сценарий рекомендаций без утечек по времени. Обучаемся на прошлом, валидируемся на более новом временном окне, финально тестируемся на самом свежем разрезе. Задача в валидации и тесте - предсказать следующий элемент (LOO). Для этого у каждого пользователя берём его последнюю интеракцию в соответствующем окне как цель, а входом служит история до неё.

__Получающиеся выборки:__
- `train` - всё до `T_val` (очищено базовыми фильтрами).
- `val_input + val_target` - окно `[T_val, T_test)`: последняя интеракция пользователя - таргет; вход - вся история до неё + история из `train`.
- `test_input + test_target` - окно `[T_test, +inf)`: тестовый набор, аналогично валидации.
- _(опц.)_ `train_val` - промежуточный слой после первого временного сплита.

### Пайплайн
1) __Первый временной сплит - выделяем тест__.\
Разбиваем исходный набор данных `ratings`, используя `TimeSplitter`, на два множества - `train_val` и `test_holdout`.
- __Зачем:__ выделить тестовую выборку.
- __Результат:__ `train_val` - данные для обучения и валидации, `test_holdout` - данные для теста.
2) __Второй временной сплит - выделяем валидацию__.\
Разбиваем получившийся с предыдущего шага набор данных `train_val`, используя `TimeSplitter`, на обучающую и валидационную выборки - `train` и `val_holdout` соответственно.
- __Зачем:__ выделить валидационную выборку.
- __Результат:__ `train` - данные для обучения, `val_holdout` - данные для валидации.
3) __Очистка обучающей части__\
_Примечение:_ ниже будет описан пример того, как может выглядеть очистка обучающей части, на практике могут быть другие варианты.\
Фильтруем из `train` последовательности длины 1.
- __Зачем:__ обучение на последовательностях длины 1 не имеет смысла, так как невозможно корректно определить историю пользователя и следующий айтем (таргет).
- __Результат:__ обучающая выборка содержит корректные последовательности, которые могут быть использованы для обучения модели.
4) __Удаление "холодных" айтемов из holdout'ов__\
Из `val_holdout` и `test_holdout` удаляются холодные айтемы с помощью функции `filter_cold`.
- __Зачем:__ для моделей, которые не умеют работать с "холодными" айтемами (например `SASRec`), оценка на таких айтемах не имеет смысла, поэтому из валидационной и тестовой выборок их необходимо убрать. Приэтом, если модель способно предсказывать "холодные" айтемы, то данный шаг можно не выполнять.
- __Результат:__ `val_holdout` и `test_holdout` содержат только те айтемы, которые содержатся в `train`. и которые модель действительно способна предсказать.
5) __Формируем LOO в валидации__
Делаем LOO сплит для валидационного набора `val_holdout`.
- __Зачем:__ последний айтем из валидации делаем таргетом, а всё до него - входом, для которого модель должна предсказать следующий элемент в истории.
- __Результат:__ `val_target` - последний айтем в валидационной выборке, `val_input` - история до последнего элемента из того же набора данных.
6) __Добавляем обучающую историю в вход валидации__\
После предыдущего шага `val_input` содержит только историю пользователя из валидационного набора `val_holdout`, приэтом данные из обучающей выборки `train` сюда не включены. Поэтому, чтобы сделать историю пользователя более полной, а не ограничиваться только лишь тем, что папало в валидацию, необходимо добавить данные из обучающего набора `train`.
- __Зачем:__ модель "видит" всю историю пользователя на момент предсказания.
- __Результат:__ `val_input` становится объединением `val_input` с предыдущего шага и `train`.
7) __Удаляем "холодных пользователей" из таргета валидации__\
После применения различных фильтров над обучающей выборкой, из неё часть пользователей, могла пропасть. Например, в нашем случае это пользователи с историей, которая содержит меньше двух айтемов. Делать оценку модели на таких пользователях бессмысленно, поскольку для них нет нупустой истории.
- __Зачем:__ корректная оценка модели для пользователей с непустой историей.
- __Результат:__ в `val_target` содержатся только те пользователи, которые есть в `val_input`.
8) __Ограничение размера валидации__\
Ограничиваем число пользователей в валидации до 5000 (порого можно подбирать индивидуально), чтобы снизить время, требуемое для оценки на валидационном периоде.
- __Зачем:__ ускорить оценку модели.
- __Результат:__ количество юзеров в `val_target` не превышает 5000.
9) __Аналогичные шаги для теста__.


In [3]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

from typing import Literal, Sequence, Optional, Tuple
from rs_datasets import MovieLens

from replay.splitters import (
    TimeSplitter,
    LastNSplitter,
    NewUsersSplitter
)
from replay.preprocessing.filters import (
    InteractionEntriesFilter
)

## Utils

Ниже реализованы две вспомогательные функции - `filter_cold` для фильтрации холодных пользователей и айтемов, а также `merge_subsets` - для объединения наборов данных.

In [7]:
def print_metrics_table(
    metrics: pd.DataFrame,
    name: str,
    query_column: str = "user_id",
    item_column: str = "item_id",
    timestamp_column: str = "timestamp"
):
    """
    Вспомогательная функция для вывода статистики по датасету.
    """
    cnt_users = metrics[query_column].nunique()
    cnt_items = metrics[item_column].nunique()
    cnt_interactions = len(metrics)
    max_timestamp = metrics[timestamp_column].max()
    min_timestamp = metrics[timestamp_column].min()
    
    print(f"Dataset: {name}")
    print(f"Users: {cnt_users}, Items: {cnt_items}, Interactions: {cnt_interactions}, Max timestamp: {max_timestamp}, Min timestamp: {min_timestamp}\n")
     

In [8]:
def filter_cold(
    target: pd.DataFrame,
    reference: pd.DataFrame,
    *,
    mode: Literal["items", "users", "both"] = "items",
    query_column: str = "user_id",
    item_column: str = "item_id",
    copy: bool = True
):
    """
    Фильтрует целевой датасет по каталогу (айтемам) и/или пользователям,
    разрешённым в эталонном датасете.

    Параметры
    ---------
    target : pd.DataFrame
        Датасет, который требуется отфильтровать.
    reference : pd.DataFrame
        Датасет-эталон: из него берутся допустимые `item_id` и/или `user_id`.
    mode : {"items", "users", "both"}, default "items"
        Что фильтровать: только айтемы, только пользователей или оба множества.
    query_column : str, default "user_id"
        Имя столбца с пользователями.
    item_column : str, default "item_id"
        Имя столбца с айтемами.
    copy : bool, default True
        Если True — возвращает копию; если False — фильтрует на месте.

    Возвращает
    ----------
    pd.DataFrame
        Отфильтрованный датасет `target`.
    """
    if mode not in {"items", "users", "both"}:
        raise ValueError("mode must be 'items' | 'users' | 'both'")

    df = target.copy(deep=True) if copy else target

    if mode in {"items", "both"}:
        if item_column not in df.columns or item_column not in reference.columns:
            raise KeyError(f"Column '{item_column}' must be in both dataframes")
        allowed_items = reference[item_column].unique()
        df = df[df[item_column].isin(allowed_items)]

    if mode in {"users", "both"}:
        if query_column not in df.columns or query_column not in reference.columns:
            raise KeyError(f"Column '{query_column}' must be in both dataframes")
        allowed_users = reference[query_column].unique()
        df = df[df[query_column].isin(allowed_users)]

    return df
        

In [9]:
def merge_subsets(
    *dfs: pd.DataFrame,
    columns: Optional[Sequence[str]] = None,
    check_columns: bool = True,
    subset_for_duplicates: Optional[Sequence[str]] = None,
    on_duplicate: Literal["error", "drop", "ignore"] = "error",
):
    """
    Объединяет несколько датафреймов построчно с согласованием столбцов
    и контролем дубликатов.

    Параметры
    ---------
    dfs : pd.DataFrame
        Перечень датафреймов для объединения.
    columns : Sequence[str] | None, default None
        Явный порядок/набор столбцов. Если None — берётся порядок из первого df.
    check_columns : bool, default True
        Проверять совпадение множеств столбцов у всех датафреймов.
    subset_for_duplicates : Sequence[str] | None, default None
        Подмножество столбцов для поиска дубликатов; по умолчанию — все.
    on_duplicate : {"error", "drop", "ignore"}, default "error"
        Политика обработки дубликатов.

    Возвращает
    ----------
    pd.DataFrame
        Объединённый датафрейм без дубликатов (если выбрано `drop`).
    """
    if not dfs:
        raise ValueError("At least one dataframe is required")
    
    ref_cols = list(dfs[0].columns) if columns is None else list(columns)
        
    # Проверка на совпадение столбцов
    aligned = []
    for i, df in enumerate(dfs):
        if check_columns:
            if set(df.columns) != set(ref_cols):
                raise ValueError(
                    f"Columns mismatch in dataframe #{i}: "
                    f"{sorted(df.columns)} != {sorted(ref_cols)}"
                )
        aligned.append(df[ref_cols])

    merged = pd.concat(aligned, axis=0, ignore_index=True)
            
    # Удаление дубликатов
    dup_subset = ref_cols if subset_for_duplicates is None else list(subset_for_duplicates)
    dup_mask = merged.duplicated(subset=dup_subset, keep="first")
    dup_count = int(dup_mask.sum())
    
    if dup_count > 0:
        if on_duplicate == "error":
            sample = merged.loc[dup_mask, dup_subset].head(5)
            raise ValueError(
                f"Found {dup_count} duplicate rows on subset {dup_subset}. "
                f"Sample:\n{sample}"
            )
        if on_duplicate == "drop":
            merged = merged.drop_duplicates(subset=dup_subset, keep="first").reset_index(drop=True)

    return merged

## Pipeline

In [10]:
ml = MovieLens("1m")  
ratings = ml.ratings      

query_column = "user_id"
item_column = "item_id"

print_metrics_table(ratings, "ratings")

Dataset: ratings
Users: 6040, Items: 3706, Interactions: 1000209, Max timestamp: 1046454590, Min timestamp: 956703932



In [11]:
time_splitter = TimeSplitter(
    time_threshold=0.1,
    query_column=query_column,
    item_column=item_column
)

interaction_filter = InteractionEntriesFilter(
    min_inter_per_user=2,
    query_column=query_column,
    item_column=item_column
)

loo_splitter = LastNSplitter(
    N=1,
    divide_column=query_column,
    query_column=query_column,
    item_column=item_column
)

Создание тестового holdout

In [12]:
train_val, test_holdout = time_splitter.split(ratings)
print_metrics_table(train_val, "train_val")
print_metrics_table(test_holdout, "test_holdout")

Dataset: train_val
Users: 6011, Items: 3678, Interactions: 900188, Max timestamp: 978133367, Min timestamp: 956703932

Dataset: test_holdout
Users: 1209, Items: 3407, Interactions: 100021, Max timestamp: 1046454590, Min timestamp: 978133414



Создание валидационного holdout

In [13]:
train, val_holdout = time_splitter.split(train_val)
print_metrics_table(train, "train")
print_metrics_table(val_holdout, "val_holdout")

Dataset: train
Users: 5454, Items: 3662, Interactions: 810169, Max timestamp: 975965591, Min timestamp: 956703932

Dataset: val_holdout
Users: 1045, Items: 3278, Interactions: 90019, Max timestamp: 978133367, Min timestamp: 975965621



Очистка обучающей части от слишком коротких последовательностей

In [14]:
train = interaction_filter.transform(train)
print_metrics_table(train, "train")

Dataset: train
Users: 5454, Items: 3662, Interactions: 810169, Max timestamp: 975965591, Min timestamp: 956703932



Число данных в обучающей выборке не изменилось, поскольку набор не содержит пользователей, для которых есть только одно взаимодействие.

__Примечание:__
После разбиения обучающая выборка часто меняется: отделяется валидация, применяются различные фильтры (min count, удаление дубликатов, фильтры последовательностей и т.д.). Итоговая отфильтрованная обучающая выборка будет не равна, той, которая была на момент сплита, поэтому в `test_holdout` будут новые холодные айтемы/пользователи. Исходя из этого, более логично фильтрацию холодных айтемов/пользователей вынести в отдельную функцию, чем оставлять внутри класса сплиттера.

Удаление "холодных" айтемов из holdout'ов на основании очищенного обучающего набора. 

In [15]:
val_holdout = filter_cold(val_holdout, train, mode="items")
test_holdout = filter_cold(test_holdout, train, mode="items")

print_metrics_table(val_holdout, "val_holdout")
print_metrics_table(test_holdout, "test_holdout")

Dataset: val_holdout
Users: 1045, Items: 3262, Interactions: 90002, Max timestamp: 978133367, Min timestamp: 975965621

Dataset: test_holdout
Users: 1208, Items: 3374, Interactions: 99923, Max timestamp: 1046454590, Min timestamp: 978133414



__Опционально:__ удаление холодных айтемов из `train_val`.

In [16]:
train_val = filter_cold(train_val, train, mode="items")
print_metrics_table(train_val, "train_val")

Dataset: train_val
Users: 6011, Items: 3662, Interactions: 900171, Max timestamp: 978133367, Min timestamp: 956703932



Формирование валидационного таргета и входа, отделяя последний элемент в истории пользователя.

In [17]:
val_input, val_target = loo_splitter.split(val_holdout)
print_metrics_table(val_input, "val_input")
print_metrics_table(val_target, "val_target")

Dataset: val_input
Users: 987, Items: 3257, Interactions: 88957, Max timestamp: 978133348, Min timestamp: 975965621

Dataset: val_target
Users: 1045, Items: 692, Interactions: 1045, Max timestamp: 978133367, Min timestamp: 975966585



Сборка входа для валидации (добавление истории из обучающего набора).

In [18]:
val_input = merge_subsets(train, val_input)
print_metrics_table(val_input, "val_input")

Dataset: val_input
Users: 6011, Items: 3662, Interactions: 899126, Max timestamp: 978133348, Min timestamp: 956703932



Удаление "холодных" пользователей: оставляем в `val_target` только тех пользователей, которые присутствуют в `val_input`.

In [19]:
val_target = filter_cold(val_target, val_input, mode="users")
print_metrics_table(val_target, "val_target")

Dataset: val_target
Users: 1045, Items: 692, Interactions: 1045, Max timestamp: 978133367, Min timestamp: 975966585



Ограничение размера валидации (5000 пользователей)

In [20]:
if val_target[query_column].nunique() > 5000:
    new_users_splitter = NewUsersSplitter(
        test_size=5000/val_target[query_column].nunique(),
        query_column=query_column,
        item_column=item_column,
    )
    _, val_target = new_users_splitter.split(val_target)
    
print_metrics_table(val_target, "val_target")

Dataset: val_target
Users: 1045, Items: 692, Interactions: 1045, Max timestamp: 978133367, Min timestamp: 975966585



По аналогии с валидацией делаем обработку для теста.

In [21]:
test_input, test_target = loo_splitter.split(test_holdout)

test_input = merge_subsets(train, test_input)

test_target = filter_cold(test_target, test_input, mode="users")

if test_target[query_column].nunique() > 10000:
    new_users_splitter = NewUsersSplitter(
        test_size=10000/test_target[query_column].nunique(),
        query_column=query_column,
        item_column=item_column,
    )
    _, test_target = new_users_splitter.split(test_target)
    
print_metrics_table(test_target, "test_target")

Dataset: test_target
Users: 1203, Items: 788, Interactions: 1203, Max timestamp: 1046454590, Min timestamp: 978136554

